In [ ]:
from itertools import product

In [ ]:
def tableaux(n, k):
    min_tab = list(range(1, n*k, n))
    offsets = product(range(n), repeat=k)
    # print(list(offsets))
    for offset in offsets:
        for idx in range(1, k):
            if offset[idx] == 0 and offset[idx - 1] == n - 1:
                break
        else:
            yield tuple(m + o for m, o in zip(min_tab, offset))

In [ ]:
len(list(tableaux(3,3)))

In [ ]:
def covers(tab1, tab2):
    diffs = [t2 - t1 for t1, t2 in zip(tab1, tab2)]
    if all(diff >= 0 for diff in diffs) and sum(abs(diff) for diff in diffs) == 1:
        # do colors
        return True
    return False

The below cell implements the algorithm from section 3 of:

Eppstein, David. "Algorithms for drawing media." International Symposium on Graph Drawing. Springer, Berlin, Heidelberg, 2004.
https://link.springer.com/content/pdf/10.1007/978-3-540-31843-9_19.pdf

Note that the equations for X_i and Y_i on page 176 actually compute X_i + 1, Y_i + 1 (at least as given here.) As best I can figure, if implemented as written in the paper, the vectors become identically zero.

In [ ]:
def basis(tabl, N):
    def dot(v, w):
        return sum(vv * ww for vv, ww in zip(v, w))
    k = len(tabl[0])
    
    X = [None for _ in range(k)]
    X[0] = 0
    for i in range(1, k):
        running_max_X = None
        for j in range(1, N):
            this_slice = [x for x in tabl if x[i - 1] == j]
            last_slice = [x for x in tabl if x[i - 1] == j - 1]
            if this_slice and last_slice:
                xp = min(this_slice, key=lambda p: dot(p[:i], X[:i]))
                xq = max(last_slice, key=lambda q: dot(q[:i], X[:i]))
                this_max_X = dot(xp[:i], X[:i]) - dot(xq[:i], X[:i])
                
                if running_max_X:
                    running_max_X = max(this_max_X, running_max_X)
                else:
                    running_max_X = this_max_X
        X[i] = -1 + running_max_X

    Y = [None for _ in range(k)]
    Y[-1] = 0
    for i in range(k-1, 0, -1):
        running_max_Y = None
        for j in range(1, N):
            this_slice = [x for x in tabl if x[i - 1] == j]
            last_slice = [x for x in tabl if x[i - 1] == j - 1]
            if this_slice and last_slice:
                yp = min(this_slice, key=lambda p: dot(p[i:], X[i:]))
                yq = max(last_slice, key=lambda q: dot(q[i:], X[i:]))
                this_max_Y = dot(yp[i:], X[i:]) - dot(yq[i:], X[i:])
                
                if running_max_Y:
                    running_max_Y = max(this_max_Y, running_max_Y)
                else:
                    running_max_Y = this_max_Y
        Y[i-1] = -1 + running_max_Y

    return (X, Y)

The below cell has the goods.

It should generate a pdf.

In [ ]:
N = 3
K = 4

lNK = list(tableaux(N, K))

# Matrix-vector multiplication
def mult(M, v):
    return [sum(vv * ww for vv, ww in zip(v, w)) for w in M]

B = list(basis(lNK, N * K + 1))
B[0] = [-1 * a for a in B[0]]
# print(list(zip(B)))
print(B)

# I was playing around with trying to rotate it here,
# but need homogeneous coordinates and ugh
th = 0
rotate = [[cos(th), sin(th)], (-sin(th), cos(th))]
mult(rotate, mult(B, lNK[0]))

pos = {v: mult(rotate, mult(B, v)) for v in lNK}

G = Graph([lNK, covers], pos=pos)
G.set_latex_options(graphic_size=(5*B[0][-1]*K, 5*B[0][-1]*K), vertex_size=0.1,
                    edge_thickness=0.05, vertex_labels=True)
view(G)

In [ ]:
# Verifying that, as in Eppstein, we get X-coordinates that
# are powers of 2 for a hypercube.
def hyper(d):
    return list(product([0, 1], repeat=d))
basis(hyper(10),2)

In [4]:
weights = var('a b c d e f g h i j k l m n o p q r s t u v w x y z A B C D E F G H I J K L M N O P Q R S T U V W X Y Z Xp alpha beta gamma delta epsilon zeta kappa nu phi eta lmbda mu')

solve([a+c==2, b+d==2,
      g-a==0, e+k==3,
      f+i==3, m-b==0,
      h-c==0, j+o==3,
      l+p+q==3, n-d==0,
      s-f==1, r+w-e==1,
     -g-h==-2, t+z==4,
      u+v-i==1, C-j==1,
      A+G-l==1, x-k==1,
      y+D+I==4, -m-n==-2,
      B+K-p==1, E+F-o==1,
     H+L-q==1, J==1,
     M==2, Q-r==-1,
     O-s-u==-1, N+S-t==2,
     P-v==-1, Xp==2,
     T+beta-y==2, R-w-x==-1,
     epsilon-A-B==-1, U+W-z==2,
     V+X+nu-D==2, Z-E-C==-1,
     Y+kappa==2, alpha-F==-1,
     zeta-G-H==-1, gamma==2,
     delta+phi-I==2, -J==-1,
     eta-K-L==-1, lmbda+mu==2,
      a==e*f/b, r==M*N/s, f==s*u/i],
      weights,solution_dict=True)

TypeError: unable to make sense of Maxima expression '[if(((c708*(sqrt(c710^2-10*c710+9)-c710+3))/(2*(2-c706))!=0)and(3-(sqrt(c710^2-10*c710+9)-c710+3)/2!=0)and((sqrt(c710^2-10*c710+9)-c710+3)/2+1!=0),[_SAGE_VAR_A==c690,_SAGE_VAR_B==c691,_SAGE_VAR_C==c692,_SAGE_VAR_D==c693,_SAGE_VAR_E==c694,_SAGE_VAR_F==(-c694)-c692+5,_SAGE_VAR_G==c695,_SAGE_VAR_H==c696,_SAGE_VAR_I==c697,_SAGE_VAR_J==1,_SAGE_VAR_K==c698,_SAGE_VAR_L==(-c698)-c696-c695-c691-c690+6,_SAGE_VAR_M==2,_SAGE_VAR_N==(((c699+1)*(sqrt(c710^2-10*c710+9)-c710+3))/2+c699+1)/2,_SAGE_VAR_O==(2*(sqrt(c710^2-10*c710+9)-c710+3))/((sqrt(c710^2-10*c710+9)-c710+3)/2+1),_SAGE_VAR_P==(3-(sqrt(c710^2-10*c710+9)-c710+3)/2)/((sqrt(c710^2-10*c710+9)-c710+3)/2+1),_SAGE_VAR_Q==c699,_SAGE_VAR_R==3-c699,_SAGE_VAR_S==c700,_SAGE_VAR_T==c701,_SAGE_VAR_U==c702,_SAGE_VAR_V==c703,_SAGE_VAR_W==((-((c699+1)*(sqrt(c710^2-10*c710+9)-c710+3))/2)-2*c702-2*c700-c699+15)/2,_SAGE_VAR_X==c704,_SAGE_VAR_Xp==2,_SAGE_VAR_Y==c705,_SAGE_VAR_Z==c694+c692-1,_SAGE_VAR_a==2-c

In [ ]:
weights2 = var("e k x w r R Q")

In [ ]:
solve([e*k==w*x, r*w==Q*R, e+k==3, r+w-e==1, x-k==1, Q+R==3, Q-r==-1, R-w-x==-1],weights2)

In [ ]:
weights3=var("j o C E F Z alpha")

In [ ]:
solve([j*C==o*E,E*F==Z*alpha,j+o==3,C-j==1, E+F-o==1, Z-E-C==-1,-Z-alpha==-3,alpha-F==-1],weights3)

In [ ]:
weights4=var("f i s u v O P")

In [ ]:
solve([f*i==s*u, u*v==O*P, s-f==1, u+v-i==1, O-s-u==-1, f+i==3, P-v==-1,-O-P==-3],weights4)

In [ ]:
weights=var("a b c d e f g h i j k l m n o p q r s")
for w in weights:
    assume(w, "rational")
    assume(w>0)

In [ ]:
solve([a+c==2,e-a==0,h-c==0,-e-h==-2,f+g==3,l-f==1,o+p-g==1,r==1,b==1,d==2,-k==-2,i+j==2,q-i==0,s-j==0,-q-s==-2,m+n==3, f==a*b/d, m==d*e/l, i==b*c/g,o==f*g/l],weights)

In [ ]:
assumptions()

In [20]:
weights_cube = var("a b c d e f g h i j k l")
solve([a+b+c==3,
       d+e-a==1, f+g-b==1, h+i-c==1,
       j-d-f==-1, k-e-h==-1, l-g-i==-1,
       -j-k-l==-3,l==f*h/j],weights_cube)

[a + b + c == 3, -a + d + e == 1, -b + f + g == 1, -c + h + i == 1, -d - f + j == -1, -e - h + k == -1, -j - k - l == -3, l == f*h/j, -g - i + l == -1]

In [61]:
R.<a,b,c,d,e,f,g,h,i,j,k,l> = QQ[]

In [65]:
R.inject_variables()

Defining a, b, c, d, e, f, g, h, i, j, k, l


In [66]:
J = R.ideal(a+b+c-3,
       d+e-a-1, f+g-b-1, h+i-c-1,
       j-d-f+1, k-e-h+1, l-g-i+1,
       -j-k-l+3,a*c-e*h,a*b-f*d,d*e-j*k,h*i-k*l,f*g-j*l,b*c-g*i)

In [70]:
J.dimension()
J.groebner_basis()

[i^2*l + 2*i*k*l + k^2*l - 4*i^2 - 4*k*l + 4*i, f*h + f*l + h*l + l^2 - 4*l, f*i - f*l - k*l - l^2 - f + 3*l, h*i - k*l, f*k + f*l + i*l + 2*k*l + l^2 + f - 4*i - 3*l, h*k + i*l + 2*k*l - 4*h - 4*i + 4, a + f + h + l - 4, b - f + i - l, c - h - i + 1, d + f + k + l - 4, e + h - k - 1, g + i - l - 1, j + k + l - 3]

In [81]:
JS = S.ideal(a+b-3,c+d-a-1,e-b-1,f-c+1,g-d-e+1,-f-g+3,a*b-d*e,c*d-f*g)
JS.dimension()

1

In [62]:
S.<a,b,c,d,e,f,g> = QQ[]

In [79]:
JS.variety()

[{b: -7/50, c: 200/43, a: 157/50, f: 157/43, g: -28/43, d: -1099/2150, e: 43/50}]

In [16]:
L33 = PolynomialRing(QQ, [char for char in 'abcdefghijklmnopqrs'])
L33.inject_variables()
S = L33.ideal(a+c-2,e-a,h-c,-e-h+2,f+g-3,l-f-1,o+p-q-1,r-1,
              b-1,d-2,-k+2,i+j-2,q-i,s-j,-q-s+2,m+n-3,
              d*f-a*b,d*e-l*m,b*c-g*i,f*g-l*o,h*j-n*r)
S.dimension()

Defining a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s


0

In [17]:
S.variety()

[{i: 1/2, b: 1, c: 4/3, n: 2, o: 2/3, a: 2/3, l: 4/3, m: 1, f: 1/3, g: 8/3, r: 1, s: 3/2, d: 2, e: 2/3, p: 5/6, q: 1/2, j: 3/2, k: 2, h: 4/3}]

In [114]:
set_verbose(2)

In [ ]:
L34 = PolynomialRing(QQ, [char for char in 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'] + ['Xp','alpha','beta','gamma','delta','epsilon','zeta', 'kappa', 'nu', 'phi', 'eta', 'lmbda', 'mu'])
L34.inject_variables()

R1 = L34.ideal(a+c-2, g-a-0, h-c-0,-g-h+2, a*c-g*h)
B1 = L34.ideal(b+d-2, m-b-0, n-d-0,-m-n+2, b*d-m*n)
R2 = L34.ideal(f+i-3, s-f-1, u+v-i-1, O-s-u+1, P-v+1, -O-P+3, f*i-s*u, u*v-O*P)
B2 = L34.ideal(e+k-3, r-w-e-1, x-k-1, Q-r+1, R-w-x+1, -Q-R+3, r*w-Q*R, e*k-x*w)
R3 = L34.ideal(l+p+q-3, A+G-l-1, B+K-p-1, H+L-q-1, epsilon-A-B+1, zeta-G-H+1, eta-K-L+1, epsilon+eta+zeta-3)
B3 = L34.ideal(j+o-3, E+F-o-1, C-j-1, Z-E-C+1, alpha-F+1, -Z-alpha+3, E*F-Z*alpha, j*o-C*E)
R4 = L34.ideal(M-2)
B4 = L34.ideal(J-1)
R5 = L34.ideal(y+D+I-4, T+beta-y-2, V+X+nu-D-2,)
B5 = L34.ideal(t+z-4,N+S-t-2,U+W-z-2, t*z-S*U)
links = [a*b-e*f, e*g-s*t, b*c-i*j, l*n-x*y, n*p-D*E, r*S-M*N, n*q-J*I, v*C-X*Xp]
RB = L34.ideal(sum([I.groebner_basis() for I in [R1,B1,R2,B2,R3,B3,R4,B4,R5,B5]],links))
rRB = L34.ideal(RB.interreduced_basis())

list(rRB.basis)
verbose(rRB.dimension(),level=2)

In [1]:
S = L34.ideal(a+c-2, b+d-2,
      g-a-0, e+k-3,
      f+i-3, m-b-0,
      h-c-0, j+o-3,
      l+p+q-3, n-d-0,
      s-f-1, r+w-e-1,
     -g-h+2, t+z-4,
      u+v-i-1, C-j-1,
      A+G-l-1, x-k-1,
      y+D+I-4, -m-n+2,
      B+K-p-1, E+F-o-1,
     H+L-q-1, J-1,
     M-2, Q-r+1,
     O-s-u+1, N+S-t-2,
     P-v+1, Xp-2,
     T+beta-y-2, R-w-x+1,
     epsilon-A-B+1, U+W-z-2,
     V+X+nu-D-2, Z-E-C+1,
     Y+kappa-2, alpha-F+1,
     zeta-G-H+1, gamma-2,
     delta+phi-I-2, -J+1,
     eta-K-L+1, lmbda+mu-2,     a*b-e*f, a*d-k*l,  b*c-i*j, c*d-o*p,
     e*g-s*t, e*k-w*x)
f = singular(S).facstd()

Defining a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z, A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, V, W, X, Y, Z, Xp, alpha, beta, gamma, delta, epsilon, zeta, kappa, nu, phi, eta, lmbda, mu


[1]:
   _[1]=lmbda+mu-2
   _[2]=epsilon+zeta+eta-3
   _[3]=gamma-2
   _[4]=Xp-2
   _[5]=Z+alpha-3
   _[6]=Y+kappa-2
   _[7]=T+V+X+beta+delta+nu+phi-10
   _[8]=Q+R-3
   _[9]=O+P-3
   _[10]=N+S+U+W-8
   _[11]=M-2
   _[12]=K+L-eta-1
   _[13]=J-1
   _[14]=I-delta-phi+2
   _[15]=G+H-zeta-1
   _[16]=F-alpha-1
   _[17]=D-V-X-nu+2
   _[18]=C+E-Z-1
   _[19]=A+B-epsilon-1
   _[20]=z-U-W+2
   _[21]=y-T-beta+2
   _[22]=w+x-R-1
   _[23]=v-P-1
   _[24]=t-N-S+2
   _[25]=s+u-O-1
   _[26]=r-Q-1
   _[27]=q-H-L+1
   _[28]=p-B+L-eta
   _[29]=o-E-alpha
   _[30]=m+n-2
   _[31]=l+B+H-epsilon-zeta-1
   _[32]=k-x+1
   _[33]=j+E-Z
   _[34]=i-u-P
   _[35]=g+h-2
   _[36]=f+u-O
   _[37]=e+x-Q-R-1
   _[38]=d-n
   _[39]=c-h
   _[40]=b+n-2
   _[41]=a+h-2
   _[42]=x*R-4*x+4
   _[43]=2*h*R-4*u*R+B*R+H*R-4*P*R+R*eta-8*h+12*u+12*P+4*R-20
   _[44]=u*E-B*E+E*L+E*P+u*alpha-B*alpha+L*alpha+P*alpha-E*eta-alpha*eta+2*h-3*u-3*P
   _[45]=x*B-B*E+x*H+E*L-B*alpha+L*alpha+x*eta-E*eta-alpha*eta+2*n-4*x-B-H-eta+4
   _[46]=u*x-B*E+E*L